# 🤖 Nano.Ai APK Builder

Build the Nano.Ai Android APK directly in Google Colab!

**Runtime:** ~15-20 minutes on first run

**Instructions:** Run all cells in order (Runtime → Run all)

## Step 1: Install Java 17

In [ ]:
# Install Java 17
!apt-get update -qq
!apt-get install -qq openjdk-17-jdk-headless > /dev/null 2>&1

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"

!java -version
print("\n✅ Java 17 installed")

## Step 2: Install Android SDK & NDK

In [ ]:
import os

# Setup Android SDK
ANDROID_HOME = '/root/android-sdk'
os.environ['ANDROID_HOME'] = ANDROID_HOME
os.environ['ANDROID_SDK_ROOT'] = ANDROID_HOME

# Install command-line tools
!mkdir -p {ANDROID_HOME}/cmdline-tools
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip -O /tmp/cmdtools.zip
!unzip -q -o /tmp/cmdtools.zip -d {ANDROID_HOME}/cmdline-tools/
!mv {ANDROID_HOME}/cmdline-tools/cmdline-tools {ANDROID_HOME}/cmdline-tools/latest 2>/dev/null || true
!rm /tmp/cmdtools.zip

os.environ['PATH'] = f"{ANDROID_HOME}/cmdline-tools/latest/bin:{ANDROID_HOME}/platform-tools:{os.environ['PATH']}"

print("✅ Android SDK command-line tools installed")

In [ ]:
import os

# Accept licenses
!yes | sdkmanager --licenses > /dev/null 2>&1

# Install SDK components
print("Installing Android SDK components (this takes a few minutes)...")
!sdkmanager "platforms;android-36" "build-tools;36.0.0" "platform-tools" 2>&1 | tail -5

# Install NDK - try multiple versions for compatibility
print("\nInstalling NDK...")
!sdkmanager "ndk;26.1.10909125" 2>&1 | tail -3

# Install CMake
print("\nInstalling CMake...")
!sdkmanager "cmake;3.22.1" 2>&1 | tail -3

# Find actual NDK path
!ls -la /root/android-sdk/ndk/

print("\n✅ SDK components installed")

In [ ]:
import os
import glob

# Auto-detect NDK path
ndk_dirs = glob.glob('/root/android-sdk/ndk/*')
if ndk_dirs:
    NDK_HOME = sorted(ndk_dirs)[-1]  # Get latest version
    os.environ['ANDROID_NDK_HOME'] = NDK_HOME
    os.environ['ANDROID_NDK'] = NDK_HOME
    print(f"✅ NDK found: {NDK_HOME}")
    
    # Verify NDK is valid
    ndk_source = os.path.join(NDK_HOME, 'source.properties')
    if os.path.exists(ndk_source):
        with open(ndk_source) as f:
            print(f"   {f.read().strip()}")
    else:
        print("⚠️ NDK source.properties not found")
else:
    print("❌ No NDK found!")
    NDK_HOME = None

## Step 3: Clone Repository

In [ ]:
%cd /content
!rm -rf NanoAi 2>/dev/null || true
!git clone https://github.com/mithun50/NanoAi.git
%cd NanoAi
print("\n✅ Repository cloned")

In [ ]:
# Clone llama.cpp
%cd /content/NanoAi
!rm -rf app/src/main/cpp/llama.cpp 2>/dev/null || true
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git app/src/main/cpp/llama.cpp
print("\n✅ llama.cpp cloned")

In [ ]:
import os

# Create local.properties with correct paths
ANDROID_HOME = os.environ.get('ANDROID_HOME', '/root/android-sdk')
NDK_HOME = os.environ.get('ANDROID_NDK_HOME', '')

local_props = f"""sdk.dir={ANDROID_HOME}
ndk.dir={NDK_HOME}
cmake.dir={ANDROID_HOME}/cmake/3.22.1
"""

with open('/content/NanoAi/local.properties', 'w') as f:
    f.write(local_props)

print("✅ local.properties created:")
print(local_props)

## Step 4: Build APK

In [ ]:
%cd /content/NanoAi
!chmod +x gradlew
print("🔨 Starting build... (10-15 minutes)\n")
!./gradlew assembleDebug --no-daemon -Dorg.gradle.jvmargs="-Xmx4g"
print("\n✅ Build complete!")

## Step 5: Download APK

In [ ]:
import os
import glob
from google.colab import files

# Find APKs
apks = glob.glob('/content/NanoAi/app/build/outputs/apk/**/*.apk', recursive=True)

if apks:
    print("📦 Generated APKs:")
    for apk in apks:
        size_mb = os.path.getsize(apk) / (1024 * 1024)
        print(f"   - {os.path.basename(apk)} ({size_mb:.1f} MB)")
    
    # Download the debug APK
    apk_path = '/content/NanoAi/app/build/outputs/apk/debug/app-debug.apk'
    if os.path.exists(apk_path):
        !cp {apk_path} /content/NanoAi-debug.apk
        print("\n📥 Downloading NanoAi-debug.apk...")
        files.download('/content/NanoAi-debug.apk')
    else:
        print(f"\n📥 Downloading {os.path.basename(apks[0])}...")
        files.download(apks[0])
else:
    print("❌ No APK found. Check build logs above.")

---

## 📱 Installation

1. Transfer APK to your Android device
2. Enable "Install from unknown sources"
3. Install the APK
4. Download a GGUF model and import it

**Recommended Models:**
- TinyLlama 1.1B Q4_K_M (~700MB)
- Qwen 2.5 3B Q4_K_M (~2GB)
- Phi-2 Q4_K_M (~1.6GB)